In [27]:
from IPython.core.display import HTML
HTML("""
<style>
.container { width:100% !important; }
</style>
""")

### Goal of this video
* Convince you guys emperically that the CRR Binomial Tree Model for European options converges to the Black Scholes Formula as $n\rightarrow\infty$
* Note we can actually show this mathematically (If I get enough comments I can show you)
    
    **Hints**
    - Central limit theorom -> Binomial Distribution becomes Normal as n gets large
    - Expand the risk neutral probability with Taylor Series
  

Let's start off where we left off in the previous video with the binomial tree model and show the value for various $n$ steps

#### Imports

In [28]:
import numpy as np

### Inputs

In [31]:
S = 100
K = 105
sigma = 0.2
r = 0.05
T = 1
n = 100
kind = 'call'
style = 'European'

### Binomial Tree Model

In [30]:
def intrinsic_value(S,K,kind):
    assert kind in ['call','put'],"Must be call or put"
    return max(S-K,0) if kind == 'call' else max(K-S,0)
    
def binomial_tree_price(S,K,sigma,r,T,n,kind='call',style='European'):
    assert kind in ['call','put'],"Must be call or put"
    assert style in ['European','American'],"Must be European or American"
    
    h = T/n
    u = np.exp(sigma*np.sqrt(h))
    d = 1/u
    p_rn = (np.exp(r*h)-d)/(u-d)
    discount_factor = np.exp(-r*h)
    
    V = np.zeros(shape=(n+1,n+1))
    
    # At expiry (j=n) we know the option values
    for i in range(n+1):
        V[i,n]=intrinsic_value(S*u**i*d**(n-i),K,kind)
    # Go backwords in time
    for j in range(n-1,-1,-1):
        for i in range(j+1):
            continuation_value=discount_factor*(p_rn*V[i+1,j+1]+(1-p_rn)*V[i,j+1])
            if style == 'American':
                early_exercise_value = intrinsic_value(S*u**i*d**(j-i),K,kind=kind)
                V[i,j]=max(early_exercise_value,continuation_value)
            else:
                V[i,j]=continuation_value
    return V[0,0]
            
    
        
        
    
    

In [32]:
binomial_tree_price(S,K,sigma,r,T,n,kind='call',style='European')

8.026229025058432

In [33]:
binomial_tree_price(S,K,sigma,r,T,n,kind='call',style='American')

8.026229025058432

In [34]:
binomial_tree_price(S,K,sigma,r,T,n,kind='put',style='American')

8.747558705041387

In [35]:
binomial_tree_price(S,K,sigma,r,T,n,kind='put',style='European')

7.90531859763425

##### Binomial Tree European value for various n steps
* For call and put options
* n = 100,200,500,1000,10000
* Let's also round to nearest 4 decimal places to better convince you guys

In [37]:
binomial_call_prices = {}
for n in [100,200,500,1000,10000]:
    binomial_call_prices[n] = binomial_tree_price(S,K,sigma,r,T,n,kind='call',style='European')

In [39]:
binomial_call_prices

{100: 8.026229025058432,
 200: 8.02600008453047,
 500: 8.023175991863205,
 1000: 8.02106028769686,
 10000: 8.021380371469782}

In [41]:
binomial_put_prices = {}
for n in [100,200,500,1000,10000,20000]:
    binomial_put_prices[n] = binomial_tree_price(S,K,sigma,r,T,n,kind='put',style='European')

In [42]:
binomial_put_prices

{100: 7.90531859763425,
 200: 7.905089657104705,
 500: 7.9022655644360125,
 1000: 7.900149860269342,
 10000: 7.900469944031665,
 20000: 7.900478840722425}

## Black Scholes Formula

**Black-Scholes Formula**

The Black-Scholes formula is a cornerstone in options pricing, providing a theoretical value for European-style options. It relies on several key assumptions about market behavior and the underlying asset.

**Mathematical Representation**

$$
C = S_0 N(d_1) - K e^{-rT} N(d_2)
$$

$$
P = K e^{-rT} N(-d_2) - S_0 N(-d_1)
$$

where:

*   $C$: Call option price
*   $P$: Put option price
*   $S_0$: Current price of the underlying asset
*   $K$: Strike price of the option
*   $r$: Risk-free interest rate
*   $T$: Time to expiration (in years)
*   $N(x)$: Cumulative standard normal distribution function
*   $d_1 = \frac{\ln(\frac{S_0}{K}) + (r + \frac{\sigma^2}{2})T}{\sigma \sqrt{T}}$
*   $d_2 = d_1 - \sigma \sqrt{T}$
*   $\sigma$: Volatility of the underlying asset's returns

**Interpretation**

*   The formula calculates the fair price of a call or put option based on the interplay of factors like the current stock price, strike price, time to expiration, risk-free rate, and volatility. 
*   $N(d_1)$ and $N(d_2)$ represent the probabilities that the option will expire in-the-money under the risk-neutral measure.

**Assumptions**

*   The underlying asset follows a geometric Brownian motion (constant volatility).
*   No dividends are paid on the underlying asset during the option's life.
*   No transaction costs or taxes.
*   Markets are frictionless (no arbitrage opportunities).
*   The risk-free interest rate is constant.


In [43]:
from scipy.stats import norm
N = norm.cdf
def bs_price(S,K,sigma,r,T,n,kind='call'):
    assert kind in ['call','put']
    d1 = (np.log(S/K)+(r+sigma**2/2)*T)/sigma/np.sqrt(T)
    d2 = d1 - sigma*np.sqrt(T)
    return S*N(d1)-K*np.exp(-r*T)*N(d2) if kind == 'call' else K*np.exp(-r*T)*N(-d2)-S*N(-d1)

In [45]:
bs_call_price = bs_price(S,K,sigma,r,T,n,kind='call')

In [44]:
bs_put_price = bs_price(S,K,sigma,r,T,n,kind='put')

##### Compare Binomial to BS Call Prices

In [49]:
for n_steps,option_price in binomial_call_prices.items():
    print(f"Num steps {n_steps},Price {np.round(option_price,4)}")
print("BS Price",np.round(bs_call_price,4))

Num steps 100,Price 8.0262
Num steps 200,Price 8.026
Num steps 500,Price 8.0232
Num steps 1000,Price 8.0211
Num steps 10000,Price 8.0214
BS Price 8.0214


##### Compare Binomial to BS Put Prices

In [51]:
for n_steps,option_price in binomial_put_prices.items():
    print(f"Num steps {n_steps},Price {np.round(option_price,4)}")
print("BS Price",np.round(bs_put_price,4))

Num steps 100,Price 7.9053
Num steps 200,Price 7.9051
Num steps 500,Price 7.9023
Num steps 1000,Price 7.9001
Num steps 10000,Price 7.9005
Num steps 20000,Price 7.9005
BS Price 7.9004
